# Pancake Pridiction v2

In [97]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import datetime as dt
import pandas as pd
import numpy as np
import math
import random
import statsmodels.api as sm

pd.set_option('display.max_rows', None)

In [98]:
import os
import sys

current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
sys.path.append(project_dir) if project_dir not in sys.path else None

import pancake
psp = pancake.Prediction()

## Data Preparation

### Config

In [99]:
dataset_reconds = 300

### Fetching Data

In [100]:
round_columns = ["epoch",
                 "startTimestamp",
                 "lockTimestamp",
                 "closeTimestamp",
                 "lockPrice",
                 "closePrice",
                 "lockOracleId",
                 "closeOracleId",
                 "totalAmount",
                 "bullAmount",
                 "bearAmount",
                 "rewardBaseCalAmount",
                 "rewardAmount",
                 "oracleCalled"]

def get_history(_psp, current_epoch, back_in_time=100):
    start_epoch_history = current_epoch - 2 - back_in_time

    df_history_round = pd.DataFrame(columns=round_columns)
    for i in range(start_epoch_history, current_epoch - 1):
        df_round = _psp.get_round(i)
        df_history_round = df_history_round.append(df_round)

    df_history_round = df_history_round.sort_values('epoch', ascending=False)
    df_history_round = df_history_round.reset_index(drop=True)
    
    return df_history_round

#### Creating a dataset

In [101]:
current_epoch = psp.get_current_epoch()
df = get_history(psp, current_epoch, dataset_reconds)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled
0,43033.0,1.644210e+09,1.644211e+09,1.644211e+09,427.820892,428.453949,0.0,0.0,24.919741,16.931665,7.988076,16.931665,24.172148,True
1,43032.0,1.644210e+09,1.644210e+09,1.644211e+09,427.548462,427.820892,0.0,0.0,27.014004,15.376600,11.637403,15.376600,26.203583,True
2,43031.0,1.644210e+09,1.644210e+09,1.644210e+09,428.689667,427.548462,0.0,0.0,45.461021,30.665218,14.795802,14.795802,44.097191,True
3,43030.0,1.644210e+09,1.644210e+09,1.644210e+09,427.656219,428.689667,0.0,0.0,25.973127,13.984817,11.988310,13.984817,25.193933,True
4,43029.0,1.644209e+09,1.644210e+09,1.644210e+09,426.844360,427.656219,0.0,0.0,20.815071,10.964463,9.850608,10.964463,20.190619,True


### Adding result

In [102]:
def add_result(lockPrice, closePrice):
    if lockPrice == closePrice:
        # draw
        return 0
    elif lockPrice > closePrice:
        # bear
        return -1
    elif lockPrice < closePrice:
        # bull
        return 1
        
df["result"] = df.apply(lambda row : add_result(row["lockPrice"], row["closePrice"]), axis=1)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled,result
0,43033.0,1.644210e+09,1.644211e+09,1.644211e+09,427.820892,428.453949,0.0,0.0,24.919741,16.931665,7.988076,16.931665,24.172148,True,1
1,43032.0,1.644210e+09,1.644210e+09,1.644211e+09,427.548462,427.820892,0.0,0.0,27.014004,15.376600,11.637403,15.376600,26.203583,True,1
2,43031.0,1.644210e+09,1.644210e+09,1.644210e+09,428.689667,427.548462,0.0,0.0,45.461021,30.665218,14.795802,14.795802,44.097191,True,-1
3,43030.0,1.644210e+09,1.644210e+09,1.644210e+09,427.656219,428.689667,0.0,0.0,25.973127,13.984817,11.988310,13.984817,25.193933,True,1
4,43029.0,1.644209e+09,1.644210e+09,1.644210e+09,426.844360,427.656219,0.0,0.0,20.815071,10.964463,9.850608,10.964463,20.190619,True,1


### Maximum Consecutive Occurance

In [103]:
countDf = pd.DataFrame({'result': list(df['result'][(df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(df['result'].groupby((df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

bear_count = countDf[countDf.result == -1]['runs'].max()
bull_count = countDf[countDf.result == 1]['runs'].max()
draw_count = countDf[countDf.result == 0]['runs'].max()

print(f"Bear {bear_count} / Bull {bull_count} / Draw {draw_count}")

Bear 7 / Bull 7 / Draw nan


## Strategies

In [104]:
running_columns = ["epoch", "position", "amount", "reward", "recent_loss", "factor"]
df_running = pd.DataFrame(columns=running_columns)

In [105]:
def get_partial_history(df, current_epoch, back_in_time=6):
    df_history_round = df[(df.epoch < current_epoch) & (df.epoch >= current_epoch - back_in_time)]
    return df_history_round

    
def bet(df_running, epoch, position, amount, recent_loss, factor):
    data = [epoch, position, amount, 0, recent_loss, factor]
    temp = pd.DataFrame(data=[data], columns=running_columns)
    df_running = df_running.append(temp)
    return df_running

    
def get_round_stats(df, epoch):
    df_round = df[df.epoch == epoch]

    total_amount = (df_round["bullAmount"] + df_round["bearAmount"]).iloc[0]
    if total_amount > 0:
        bull_ratio = ((df_round["bullAmount"] / total_amount) * 100).iloc[0]
        bear_ratio = ((df_round["bearAmount"] / total_amount) * 100).iloc[0]
        bear_pay_ratio = (total_amount / df_round["bearAmount"]).iloc[0]
        bull_pay_ratio = (total_amount / df_round["bullAmount"]).iloc[0]
    else:
        bull_ratio = None
        bear_ratio = None
        bear_pay_ratio = None
        bull_pay_ratio = None

    lockPrice = df_round["lockPrice"].iloc[0]
    closePrice = df_round["closePrice"].iloc[0]

    if lockPrice == closePrice:
        # draw
        result = 0
    elif lockPrice > closePrice:
        # bear
        result = -1
    elif lockPrice < closePrice:
        # bull
        result = 1
    
    return {"total_amount": total_amount,
           "bull_ratio": bull_ratio, "bear_ratio": bear_ratio,
           "bear_pay_ratio": bear_pay_ratio, "bull_pay_ratio": bull_pay_ratio,
           "result": result}


def update_reward(df, df_running, epoch, status):
    bet_value = df_running[df_running.epoch == epoch]["amount"].iloc[0]
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    
    if status == 0:
        df_running.loc[df_running.epoch == epoch, "reward"] = -1 * bet_value
        pay_ratio = 0
    elif status == 1:
        epoch_stats = get_round_stats(df, epoch)
        if bet_position == "bull":
            pay_ratio = epoch_stats["bull_pay_ratio"]
        elif bet_position == "bear":
            pay_ratio = epoch_stats["bear_pay_ratio"]

        df_running.loc[df_running.epoch == epoch, "reward"] = (bet_value * pay_ratio) - bet_value
    
    print(f"[{epoch}] - Factor {pay_ratio} - Bet {bet_value} - Position {position}")    
    return df_running


def check_result(df, df_running, epoch):
    
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    epoch_stats = get_round_stats(df, epoch)
    
    if bet_position == "bull" and epoch_stats["result"] == 1:
        result = 1
    elif bet_position == "bear" and epoch_stats["result"] == -1:
        result = 1
    else:
        result = 0
    
    df_running = update_reward(df, df_running, epoch, result)
    return df_running, epoch_stats["result"]
    

def overview(df_running):
    total_spent = df_running.sum()["amount"]
    total_loss = abs(df_running[df_running["reward"] < 0].sum()["reward"])
    loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    estimated_win = df_running[df_running["reward"] > 0].sum()["reward"]
    win_times = df_running[df_running["reward"] > 0].count()["reward"]
    estimated_gain = df_running.sum()["reward"]
    max_spent = df_running.max()["amount"]

    last_win_epoch = df_running[df_running["reward"] > 0].max()["epoch"]
    if last_win_epoch is None or math.isnan(last_win_epoch):
        recent_loss = abs(df_running.sum()["reward"])
        recent_loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    else:
        recent_loss = abs(df_running[df_running["epoch"] > last_win_epoch].sum()["reward"])
        recent_loss_times = df_running[(df_running["epoch"] > last_win_epoch)
                                       & (df_running["reward"] < 0)].count()["reward"]

    return {"total_spent": total_spent,
            "max_spent": max_spent,
            "total_loss": total_loss,
            "loss_times": loss_times,
            "estimated_win": estimated_win,
            "win_times": win_times,
            "estimated_gain": estimated_gain,
            "recent_loss": recent_loss,
            "recent_loss_times": recent_loss_times}


### Strategies Default Settings

In [106]:
starting_epoch = df.min()["epoch"] + 20
maximum_epoch = df.max()["epoch"] - 1

# 0: Even / 1: Odd
strategy_remainder = 0
base_bet = 0.01
safe_bet = 0.2

current_epoch = starting_epoch
value = base_bet
print("Starting Epoch: ", starting_epoch)

Starting Epoch:  42753.0


### Strategy 1: Trend

In [107]:
def calculate_trend(df, current_epoch, back_in_time=6):
    df_history_round = get_partial_history(df, current_epoch, back_in_time)
    X = df_history_round.index
    X = X.astype(float)

    y = df_history_round['closePrice']
    y = y.astype(float)

    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    alpha = model.params['x1']
    return alpha

### Strategy 2: Same-Before

### Strategy 3: Random

### Strategy 4: Exponential Moving Average (EMA)

In [108]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

In [109]:
while current_epoch < maximum_epoch:
    if current_epoch % 2 == strategy_remainder:

        bet_status = overview(df_running)
        current_round_stats = get_round_stats(df, current_epoch)

        df_history_round = get_partial_history(df, current_epoch, back_in_time=21)

        currentPrice = np.round(df_history_round["closePrice"].iloc[0],1)
        ema9 = np.round(calculate_ema(df_history_round["closePrice"].iloc[:9],9),1)
        ema21 = np.round(calculate_ema(df_history_round["closePrice"].iloc[:21],21),1)

        print('current price:', currentPrice)
        print('ema9 :', ema9, ' ema21:',ema21)
            
        if (ema9 == ema21):
            print('do nothing')
            position = "skip"

        elif (ema9 < ema21):
            custom_factor = current_round_stats["bear_pay_ratio"] - safe_bet
            position = "bear"
            if(currentPrice > ema9):
                print('bet bearish , confidence 100')
            else :
                print('bet bearish , confidence enough')

        elif (ema9 > ema21):
            custom_factor = current_round_stats["bull_pay_ratio"] - safe_bet
            position = "bull"
            if(currentPrice < ema9):
                print('bet bullish , confidence 100')
            else :
                print('bet bullish , confidence enough')
        
        value = (bet_status["recent_loss"] + base_bet) / (custom_factor - 1)
        if value < base_bet:
            value = base_bet
            
        if position == "bull" or position == "bear":
            df_running = bet(df_running, current_epoch, position, value, bet_status["recent_loss"], custom_factor)
            df_running, result = check_result(df, df_running, current_epoch)
        elif position == "skip":
            print(f"[{current_epoch}] - Skipped.")
            
    current_epoch += 1


current price: 415.4
ema9 : [415.1]  ema21: [416.]
bet bearish , confidence 100
[42754.0] - Factor 0 - Bet 0.014678302677070265 - Position bear
current price: 415.4
ema9 : [415.2]  ema21: [415.7]
bet bearish , confidence 100
[42756.0] - Factor 0 - Bet 0.03084932177518762 - Position bear
current price: 415.8
ema9 : [415.4]  ema21: [415.4]
do nothing
[42758.0] - Skipped.
current price: 416.1
ema9 : [415.5]  ema21: [415.2]
bet bullish , confidence enough
[42760.0] - Factor 0 - Bet 0.055277861485960575 - Position bull
current price: 415.6
ema9 : [415.6]  ema21: [415.1]
bet bullish , confidence enough
[42762.0] - Factor 2.083345413208008 - Bet 0.12543845734797093 - Position bull
current price: 415.1
ema9 : [415.6]  ema21: [415.2]
bet bullish , confidence 100
[42764.0] - Factor 0 - Bet 0.013181495439435717 - Position bull
current price: 413.7
ema9 : [415.2]  ema21: [415.2]
do nothing
[42766.0] - Skipped.
current price: 414.7
ema9 : [414.9]  ema21: [415.1]
bet bearish , confidence enough
[427

## Overview

In [110]:
sum_reward = 0
df_running = df_running.reset_index(drop=True)

for index, row in df_running.iterrows():
        reward = row["reward"]
        sum_reward += reward
        
        df_running.loc[df_running.index == index, "acc_reward"] = sum_reward
        
df_running

,epoch,position,amount,reward,recent_loss,factor,acc_reward
0,42754.0,bear,0.014678,-0.014678,0.000000,1.681278,-0.014678
1,42756.0,bear,0.030849,-0.030849,0.014678,1.799963,-0.045528
2,42760.0,bull,0.055278,-0.055278,0.045528,2.004518,-0.100805
3,42762.0,bull,0.125438,0.135893,0.100805,1.883345,0.035088
4,42764.0,bull,0.013181,-0.013181,0.000000,1.758639,0.021906
5,42768.0,bear,0.023072,-0.023072,0.013181,2.004762,-0.001165
6,42770.0,bear,0.077040,0.061661,0.036253,1.600375,0.060496
7,42772.0,bear,0.012410,0.012482,0.000000,1.805775,0.072978
8,42774.0,bear,0.010000,0.012713,0.000000,2.071283,0.085691
9,42776.0,bear,0.012170,-0.01217,0.000000,1.821663,0.073520


In [111]:
summary = overview(df_running)
print(summary)

{'total_spent': 7.331658578880165, 'max_spent': 2.1532396367326494, 'total_loss': 2.168283608912439, 'loss_times': 61, 'estimated_win': 3.907833489895577, 'win_times': 69, 'estimated_gain': 1.739549880983139, 'recent_loss': 0.0, 'recent_loss_times': 0}


### Counting Consecutive Win/Loss

In [112]:
def add_result_binary(reward):
    if reward > 0:
        # bull
        return 1
    else:
        # bear
        return -1
        
temp_df = df_running.copy()

temp_df["result"] = temp_df.apply(lambda row : add_result_binary(row["reward"]), axis=1)

countDf = pd.DataFrame({'result': list(temp_df['result'][(temp_df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(temp_df['result'].groupby((temp_df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

loss_count = countDf[countDf.result == -1]['runs'].max()
win_count = countDf[countDf.result == 1]['runs'].max()

print(f"Loss {loss_count} / Win {win_count}")

Loss 5 / Win 7
